In [33]:
import numpy as np
import librosa
from tqdm import tqdm
import pandas as pd
import os
import soundfile as sf
from scipy.io import wavfile
import parselmouth

In [60]:
t_id = '002'
split = 'test'

In [61]:
### for 002

audio_folder = f"/home/sumitk/working_dir/raaga/{split}_{t_id}"
label_folder = f"/home/sumitk/working_dir/raaga/label_{split}_{t_id}_new_copy"

In [62]:
# x, sr = librosa.load('/home/sumitk/working_dir/raaga/rod_pb_data/train/audio/BHOPALI--1346_18.wav', sr = None)

In [63]:
# x.shape, sr

In [64]:
### PB

# audio_folder = f'/home/sumitk/working_dir/raaga/sharah_surge/prasar_bharati/audio_files_vocals/{split}'
# label_folder = f'/home/sumitk/working_dir/raaga/sharah_surge/prasar_bharati/labels/{split}'

In [65]:
### combined data
# audio_folder = f'/home/sumitk/working_dir/raaga/rod_pb_data/{split}/audio'
# label_folder = f'/home/sumitk/working_dir/raaga/rod_pb_data/{split}/labels'

In [66]:
# x, s = librosa.load('/home/sumitk/working_dir/raaga/sharah_surge/prasar_bharati/audio_files_vocals/Bhairav_vocals/13971-1_1.wav', sr=None)

In [67]:
# audio_folder = f'/home/sumitk/working_dir/raaga/rod_pb_data/{split}/audio'
# label_folder = f'/home/sumitk/working_dir/raaga/rod_pb_data/{split}/labels'

In [68]:
def wav2spec(wavefile, sampling_rate,f_max, win_length, time_step):
    # Create a Parselmouth Sound object
    sound = parselmouth.Sound(wavefile, sampling_frequency=sampling_rate)

    # Create a spectrogram
    spectrogram = sound.to_spectrogram(window_length=win_length,time_step=time_step, maximum_frequency=f_max)
    return spectrogram

In [69]:
def spec2chromagram(spectrogram, sampling_rate, n_fft, hop_size,win_length, n_chroma):
    C = librosa.feature.chroma_stft(S=np.abs(spectrogram), sr=sampling_rate, n_fft = n_fft, hop_length = hop_size, win_length=win_length, n_chroma = n_chroma)
    # C = C/np.max(C)
    return C

In [70]:
def wav2chromagram(wavefile, sampling_rate, n_fft, hop_size,win_length, n_chroma):
    C = librosa.feature.chroma_stft(y=wavefile, sr=sampling_rate, n_fft = n_fft, hop_length = hop_size, win_length=win_length, n_chroma = n_chroma)
    # C = C/np.max(C)
    return C

In [71]:
def _pad_spec(spec, Fs, hop_length, clip_duration):
    T = int((clip_duration*Fs)/hop_length +1)
    if(spec.shape[1]% T != 0):
        spec = np.concatenate((spec, np.zeros((spec.shape[0], T- (spec.shape[1]% T)))), axis = 1)
    return spec

In [72]:
# def error_to_number(error_str):
#     if(error_str == 'K' or error_str =='k'):
#         error_id = 1
#     elif(error_str == 'M' or error_str == 'Me'):
#         error_id = 2
#     elif(error_str == 'Sa' or error_str == 'SA'):
#         error_id = 3
#     elif(error_str == 'An' or error_str == 'AN'):
#         error_id = 4
#     elif(error_str == 'Mu' or error_str == 'MU'):
#         error_id = 5
#     elif(error_str == 'Kh' or error_str == 'KH'):
#         error_id = 6
#     elif(error_str == 'h' or error_str == 'H'):
#         error_id = 7
#     elif(error_str == 'G' or error_str == 'g'):
#         error_id = 8
#     else:
#         error_id = 0
#     return error_id


In [73]:
def error_to_number(error_str):
    if(error_str == 'K' or error_str =='k'):
        error_id = 1
    elif(error_str == 'M' or error_str == 'Me'):
        error_id = 2
    # elif(error_str == 'Sa' or error_str == 'SA'):
    #     error_id = 3
    elif(error_str == 'An' or error_str == 'AN'):
        error_id = 3
    elif(error_str == 'Mu' or error_str == 'MU'):
        error_id = 4
    elif(error_str == 'h' or error_str == 'H'):
        error_id = 5
    elif(error_str == 'G' or error_str == 'g'):
        error_id = 6
    # elif(error_str == 'KH' or error_str == 'Kh'):
    #     error_id = 7
    else:
        error_id = 0
    return error_id

In [74]:
def save_files(files, clip_duration, Fs, hop_length, files_dir, file_id, file_name):
    # L = int((clip_duration*Fs)/hop_length +1)
    L = 568
    # print('files shape: ',files.shape)
    for q in range(0, int(files.shape[1]), L):
# #             print(i,j)
        on = int(q/56.8)
        off = int((q+L)/56.8)
        np.save(files_dir+file_name+'_'+str(on)+'_'+str(off),files[:,q:q+L])
        # np.save(files_dir+file_name+t_id+'_'+str(file_id)+'_'+str(c), files[:,q:q+L])
    return

In [75]:
def save_gt(files, clip_duration, Fs, hop_length, files_dir, file_id, file_name):
    # L = int((clip_duration*Fs)/hop_length +1)
    L = 568
    # if(len(files)% L != 0):
    #     x = np.concatenate((files, np.zeros(L- (len(files)% L))))
    # else:
    #     x = files
    for j in range(0, len(files), L):
        on = int(j/56.8)
        off = int((j+L)/56.8)
        np.save(files_dir+file_name+'_'+str(on)+'_'+str(off),files[j:j+L])
    return

In [76]:
def save_audio_clips(files, clip_duration, Fs, files_dir, file_name):
    L = int(Fs*clip_duration)
    # if(len(files)%L != 0):
    #     print('L: ',L, 'len(files): ',len(files))
    #     zeros_to_pad = np.zeros(L - len(files))
    #     files = np.concatenate((files, zeros_to_pad))
    for j in range(0, len(files), L):
        on = int(j/Fs)
        off = int((j+L)/Fs)
        np.save(files_dir+file_name+'_'+str(on)+'_'+str(off),files[j:j+L])
    return

In [ ]:
os.system(f'rm -rf ./raaga_chroma_data_{t_id}/{split}/')
os.system(f'rm -rf ./raaga_audio_data_{t_id}/{split}/')

In [ ]:
x_dir = f'./raaga_chroma_data_{t_id}/{split}/x/'           # save .npz files here
os.system('mkdir -p %s'%x_dir)
yd_dir = f'./raaga_chroma_data_{t_id}/{split}/yd/'         # save .npz files here
os.system('mkdir -p %s'%yd_dir)

In [ ]:
audio_dir = f'./raaga_audio_data_{t_id}/{split}/x/'           # save .npz files here
os.system('mkdir -p %s'%audio_dir)
audio_yd_dir = f'./raaga_audio_data_{t_id}/{split}/yd/'         # save .npz files here
os.system('mkdir -p %s'%audio_yd_dir)

In [80]:
Fs = 44100
win_length = 0.035  #in sec
n_fft = 4096
time_step = win_length/2  # in seconds 
hop_length = int(time_step*Fs)
clip_duration = 10  #in sec
num_classes = 7
n_chroma = 120
max_freq = 5000

In [81]:
# # Function to read labels from a .txt file
def read_labels_from_file(file_path):
    labels = []

    with open(file_path, 'r') as file:
        for line in file:
            onset, offset, class_name = line.strip().split('\t')
            labels.append((float(onset), float(offset), class_name))

    return labels

In [82]:
# Fs = 44100
# win_length = 0.035  #in sec
# n_fft = 4096
# time_step = win_length/2  # in seconds 
# hop_length = int(time_step*Fs)
# clip_duration = 10  #in sec
# num_classes = 7
# n_chroma = 120
# max_freq = 5000

In [83]:
# x = np.load('/home/sumitk/working_dir/raaga/raaga_audio_data_002/test/x/Bhairav_2_Alaap Bhairav_0.0_10.0.npy', allow_pickle = True)
# S = wav2spec(wavefile=x, sampling_rate=Fs,f_max=max_freq, win_length=win_length, time_step=time_step)
# print(S.values.shape)
# C = spec2chromagram(spectrogram=S.values, sampling_rate=Fs, n_fft=n_fft, hop_size=hop_length,win_length=win_length, n_chroma=n_chroma)
# print(C.shape)

In [84]:
def load_data(audio_folder_path, labels_folder_path):
    label_files = os.listdir(labels_folder_path)
    
    # audio_lessons = os.listdir(audio_folder_path)
    for i in tqdm(range(len(label_files))):
            # print('checking: ',label_files[i])
            label_file_path = os.path.join(labels_folder_path, label_files[i])
            audio_file_path = os.path.join(audio_folder_path, label_files[i].replace('txt', 'wav'))
            
            audio_wav, sr =  librosa.load(audio_file_path, sr=Fs)
            harmonic, _ = librosa.effects.hpss(audio_wav)
            label = np.array(pd.read_csv(label_file_path, header=None, delimiter='\t'))
            # print(label)
            # print('audio shape: ',audio_wav.shape)
            save_audio_clips(files = audio_wav,
                            clip_duration=10,
                            Fs = sr, files_dir = audio_dir, 
                            file_name = label_files[i].split('.')[0])
            try:
                S = wav2spec(wavefile=harmonic, sampling_rate=sr, f_max=max_freq, win_length=win_length, time_step=time_step)
                C = spec2chromagram(spectrogram=S, sampling_rate=sr, n_fft=n_fft, hop_size=hop_length, win_length=win_length, n_chroma=n_chroma)
            
                spec_padded = _pad_spec(spec = C, Fs = Fs, 
                                        hop_length = hop_length, clip_duration = clip_duration)
                # print(spec_padded.shape)
                # print(spec.shape)
                # print(label_files[i])
                gt = np.zeros(len(audio_wav))
                yd = np.zeros(int(spec_padded.shape[1]))
                for k in range(int(label.shape[0])):
                    # print(label.shape)
                    onset_time = label[k][0]
                    offset_time = label[k][1]
                    error = label[k][2]
                    
                    # print(error, type(error))
                    # print(error)
                    error_num = error_to_number(error)
                    if(str(onset_time).count('.')>1):  #convert min to sec
                        # print('enter onset')
                        time_parts = onset_time.split('.')
                        minutes = int(time_parts[0])
                        seconds = float('.'.join(time_parts[1:]))
                        onset_time = str((minutes * 60) + seconds)
                    if(str(offset_time).count('.')>1):    #convert min to sec
                        # print('enter offset')
                        time_parts = offset_time.split('.')
                        minutes = int(time_parts[0])
                        seconds = float('.'.join(time_parts[1:]))
                        offset_time = str((minutes * 60) + seconds)
                    # print(onset, offset, error)
                    # print(type(onset_time))
                    onset = int((onset_time*sr)/hop_length + 1)
                    offset = int((offset_time*sr)/hop_length + 1)
                    onset_TD = int(onset_time*sr)
                    offset_TD = int(offset_time*sr)
                    # print(onset,offset)
                    yd[onset:offset] = error_num
                    gt[onset_TD:offset_TD] = error_num
                    # print('label shape: ',gt.shape)
                    # print('error: ',error_num)
                    # yd_categorical = np.transpose(tf.keras.utils.to_categorical(yd, num_classes=num_classes, dtype='float32'))
                    # print('spec_padded shape: ',spec_padded.shape)
                    # print('yd shape: ',yd.shape)
                    # print('lf: ',label_files[i].split('.'[0]))
                    # save_audio_clips(files = gt,
                    #             clip_duration=10,
                    #             Fs = sr, files_dir = audio_yd_dir, 
                    #             file_name = label_files[i].split('.')[0])
                    save_files(spec_padded,
                                clip_duration=clip_duration, 
                                Fs=Fs, 
                                hop_length=hop_length,  
                                files_dir= x_dir, file_id=i,
                                file_name=label_files[i].split('.')[0])
                    save_gt(yd,
                                clip_duration=clip_duration, 
                                Fs=Fs, 
                                hop_length=hop_length,  
                                files_dir= yd_dir, file_id=i,
                                file_name=label_files[i].split('.')[0])
                save_audio_clips(files = gt,
                                clip_duration=10,
                                Fs = sr, files_dir = audio_yd_dir, 
                                file_name = label_files[i].split('.')[0])
                
            except Exception as e:
                print(f"Error processing {label_files[i]}: {e}")
                continue
                

    return

In [ ]:
load_data(audio_folder, label_folder)

In [ ]:
x_dir

In [28]:
# aud

In [29]:
# len(os.listdir(f'./raaga_chroma_data_PB_{t_id}/{split}/x/')), len(os.listdir(f'./raaga_chroma_data_PB_{t_id}/{split}/yd/'))

In [30]:
# import os
# import shutil

# # Define paths
# train_folder = "/home/sumitk/working_dir/raaga/train_001"
# labels_folder = "/home/sumitk/working_dir/raaga/data/labels_train_and_test_001"
# output_folder = "/home/sumitk/working_dir/raaga/train_labels_001"

# # Create output directory if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Iterate over .wav files in train folder
# for wav_file in os.listdir(train_folder):
#     if wav_file.endswith(".wav"):
#         # Get the file name without extension
#         file_name = os.path.splitext(wav_file)[0]
        
#         # Find corresponding label file in labels folder
#         label_file = file_name + ".txt"
#         label_path = os.path.join(labels_folder, label_file)
        
#         # Copy label file to output folder
#         if os.path.exists(label_path):
#             output_path = os.path.join(output_folder, label_file)
#             shutil.copy(label_path, output_path)
#             print(f"Copied {label_file} to {output_folder}")
#         else:
#             print(f"No label file found for {wav_file}")

# print("Label copying complete!")


In [31]:
# !jupyter nbconvert --to script 'dataprep_without_overlap.ipynb'